In [1]:
import os
import PyPDF2
import fitz 
import shutil
import langchain
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings 
from langchain.schema import Document 
from langchain.vectorstores.chroma import Chroma 
from langchain.chat_models import ChatOpenAI 
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

In [ ]:
file_path = r"C:/Users/Coustom Diet Plan based on blood report powered by GenAI/Blood Reports/sterling_accuris_pathology.pdf"
with open(file_path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    extracted_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        extracted_text += page.extract_text()
#print(extracted_text)
print(" ".join(extracted_text.split()))

Hemoglobin g/dL 13.0 - 16.5 Colorimetric 14.5 RBC Count million/cmm 4.5 - 5.5 Electrical impedance 4.79 Hematocrit % 40 - 49 Calculated 43.3 MCV fL 83 - 101 Derived 90.3 MCH pg 27.1 - 32.5 Calculated 30.2 MCHC g/dL 32.5 - 36.7 Calculated 33.4 RDW CV % 11.6 - 14 Calculated 13.60 Total WBC and Differential Count WBC Count H /cmm 4000 - 10000 SF Cube cell analysis 10570 Neutrophils %40 - 80Microscopic 73 Lymphocytes %20 - 40Microscopic 19 Eosinophils %1 - 6Microscopic 02 Monocytes %2 - 10Microscopic 06 Basophils %0 - 2Microscopic 00/cmm2000 - 6700 7716 /cmm1100 - 3300 2008 /cmm00 - 400 211 /cmm200 - 700 634 /cmm0 - 100 0 Platelet Count /cmm 150000 - 410000 Electrical impedance 150000 MPV H fL 7.5 - 10.3 Calculated 14.00 Peripheral Smear Examination RBC Morphology Normochromic Normocytic WBC Morphology WBCs Series Shows Normal Morphology Platelets Morphology Platelets are adequate with normal morphology. Parasites Malarial parasite is not detected. Erythrocyte Sedimentation Rate ESR mm/1hr

In [ ]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    
    return text

def extract_text_from_multiple_pdfs(pdf_directory):
    extracted_texts1 = []
    
    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, filename)
            text = extract_text_from_pdf(pdf_path)
            extracted_texts1.append(text)
    
    return extracted_texts1

pdf_directory = "C:/Users/Coustom Diet Plan based on blood report powered by GenAI/Knowledge Base"
extracted_texts1 = extract_text_from_multiple_pdfs(pdf_directory)

for i, text in enumerate(extracted_texts1):
    print(f"Text from document {i+1}:\n{text}\n")

Text from document 1:
4/16/25, 3:33 PM 
Dietary Advice For Individuals with Diabetes - Endotext - NCBI Bookshelf 
NCBI Bookshelf. A service of the National Library of Medicine, National Institutes of Health. 
Feingold KR, Ahmed SF, Anawalt B, et al., editors. Endotext [Internet]. South Dartmouth (MA): 
MDText.com, Inc.; 2000-. 
Diabetes 
Andrew Reynolds, BSc, MSc, PhD, MPH 
Senior Research Fellow, Department of Medicine, University of Otago 
Email: andrew.reynolds@otago.ac.nz 
Corresponding author. 
Joanna Mitri, MD, MS 
Chief Medical Officer at Sequel Med Tech 
Email: jmitri@sequelmedtech.com 
Last Update: April 28, 2024. 
ABSTRACT 
The chapter summarizes the current information available from a variety of scientifically based 
guidelines and resources on dietary advice for those with diabetes. It is a practical overview for health 
care practitioners working in diabetes management. The chapter is divided into sections by content 
and includes sources for further reading. A primary me

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap=110)

In [5]:
chunk1=text_splitter.create_documents(extracted_texts1)

In [6]:
chunk1

[Document(metadata={}, page_content='4/16/25, 3:33 PM \nDietary Advice For Individuals with Diabetes - Endotext - NCBI Bookshelf \nNCBI Bookshelf. A service of the National Library of Medicine, National Institutes of Health. \nFeingold KR, Ahmed SF, Anawalt B, et al., editors. Endotext [Internet]. South Dartmouth (MA): \nMDText.com, Inc.; 2000-. \nDiabetes \nAndrew Reynolds, BSc, MSc, PhD, MPH \nSenior Research Fellow, Department of Medicine, University of Otago \nEmail: andrew.reynolds@otago.ac.nz \nCorresponding author. \nJoanna Mitri, MD, MS \nChief Medical Officer at Sequel Med Tech \nEmail: jmitri@sequelmedtech.com \nLast Update: April 28, 2024. \nABSTRACT \nThe chapter summarizes the current information available from a variety of scientifically based \nguidelines and resources on dietary advice for those with diabetes. It is a practical overview for health \ncare practitioners working in diabetes management. The chapter is divided into sections by content \nand includes sources 

In [8]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [9]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
db = Chroma.from_documents(chunk1, embeddings)

In [11]:
query = "What are the best food for vitamin A defeciency?"
matching_docs = db.similarity_search(query)

matching_docs[1]

Document(metadata={}, page_content='How this can be achieved is illustrated below with reference to ﬁve micro-\nnutrients, which are considered to be of public health relevance or serve \nas markers for overall micronutrient intake. The nutrients selected for \ndiscussion include those that are among the most difﬁcult to obtain in \ncereal- and tuber-based diets (i.e. diets based on rice, corn, wheat, potato \nor cassava). Moreover, nutrient deﬁciencies of vitamin A, iron, and zinc are\nwidespread.\n17.2.1 Vitamin A\nThe vitamin A content of most staple diets can be signiﬁcantly improved with\nthe addition of a relatively small portion of plant foods rich in carotenoids,\nthe precursors of vitamin A. For example, a typical portion of cooked carrots\n(50g) added to a daily diet, or 21g of carrots per 4.184MJ, provides 500mg\nretinol equivalents, which is the recommended nutrient density for this\nvitamin. The biological activity of provitamin A varies among different plant\nsources; fru

In [12]:
# Retrieve documents from Chroma database
query = "Retrieve all documents"
retrieved_docs = db.similarity_search(query)

In [13]:
# Format the retrieved documents into a prompt
formatted_docs = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [14]:
# Create the prompt
prompt = f'''
Role: You are a powerful Diet assistant.
Please give a diet plan for the specific blood report. Here is the blood report:

{extracted_text}

Here are the WHO food recommendations:

{formatted_docs}

Instructions:
1. Diet should be based on the blood report.
2. Diet plan should be for 4 weeks.
3. Diet plan must follow WHO guidelines.
4. Diet plan should be in the form of breakfast, lunch, and dinner.
5. Food recommended in the diet plan should be very precise. If a person has low hemoglobin, then you can recommend food like jaggery, spinach, apple, etc., but also consider that you cannot recommend jaggery food to a diabetic person.
6. Don't give Important Considerations Before Starting, options, and General Principles in the output. Output should be very specific.
7. Don't add any unnecessary information in the output like Important Note, Foods to Emphasize for Iron, Foods to Emphasize for Vitamin D.
8. Don't give options in meals. Give direct food recommendations.
9. Make two diet plans: one for vegetarians and one for non-vegetarians. In the non-vegetarian diet plan, you can recommend mutton, chicken, fish, etc. Don't recommend beef.
10. Don't recommend a glass of water in every meal.
11. Food must be easily available in the Indian market.
12. Don't recommend any snacks like mid-morning snack, evening snack, etc.
13. Don't give any medical terminology in the output like Fasting Sugar, LDL Cholesterol. Just give food recommendations.
14. Output must be in this format: 4-week diet plans based on the blood report, one for vegetarians and one for non-vegetarians. The plans will focus on addressing the abnormal findings in the blood report.
    I. 4-Week Vegetarian Diet Plan:
      a. Week 1
         Breakfast:
         Lunch:
         Dinner:
      b. Week 2
         Breakfast:
         Lunch:
         Dinner:
      c. Week 3
         Breakfast:
         Lunch:
         Dinner:
      d. Week 4
         Breakfast:
         Lunch:
         Dinner:
    II. 4-Week Non-Vegetarian Diet Plan:
      a. Week 1
         Breakfast:
         Lunch:
         Dinner:
      b. Week 2
         Breakfast:
         Lunch:
         Dinner:
      c. Week 3
         Breakfast:
         Lunch:
         Dinner:
      d. Week 4
         Breakfast:
         Lunch:
         Dinner:
15. Don't add * and ** in the output.
16. Don't recommend any supplements in the diet plan like protein powder. Only give food recommendations.
17. Don't recommend any leftover food in the diet plan.
18. Don't recommend any food that is not easily available in the Indian market.
19. Don't give the same diet plan for every report. Analyze the report and give a diet plan according to WHO guidelines.
20. If you're recommending vegetable curry/vegetable biryani/vegetable salad, then also recommend the vegetable name.
21. According to the date of the blood report, give recommendations of seasonal fruits and vegetables.
22. Limit the portion size of the food. Excess of food can cause other health issues.
23. Don't recommend any food that is not easily digestible.
'''

In [ ]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="Openrouter API Key",
)

completion = client.chat.completions.create(
  #model="meta-llama/llama-4-maverick:free",
  #model="mistralai/mistral-7b-instruct:free",
  model="google/gemini-2.0-flash-exp:free",
  
  messages=[
    {
      "role": "user",
      "content": prompt
    }
  ],
  temperature=0,
  max_tokens=1000
)
print(completion.choices[0].message.content)

Here are the 4-week diet plans based on the blood report, one for vegetarians and one for non-vegetarians. The plans will focus on addressing the abnormal findings in the blood report.

I. 4-Week Vegetarian Diet Plan:
 a. Week 1
 Breakfast: Sprouted moong dal cheela with a side of papaya.
 Lunch: Palak paneer with 2 multigrain rotis and a small bowl of cucumber raita.
 Dinner: Vegetable and lentil soup (carrots, beans, peas, and masoor dal) with a small bowl of brown rice.
 b. Week 2
 Breakfast: Besan (gram flour) and vegetable (onion, tomato, capsicum) omelet with a glass of orange juice.
 Lunch: Vegetable biryani (carrots, peas, beans) with a side of mixed vegetable raita.
 Dinner: Dal makhani with 2 rotis and a side of carrot and beetroot salad.
 c. Week 3
 Breakfast: Ragi dosa with sambar and a small bowl of pomegranate.
 Lunch: Chole (chickpea) curry with 2 rotis and a side of onion and tomato salad.
 Dinner: Vegetable stew (potatoes, carrots, beans) with a small bowl of quinoa.
 

In [15]:
completion

ChatCompletion(id='gen-1744804886-LQJc1MZA06bQ5R9MUW8t', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are the 4-week diet plans based on the blood report, one for vegetarians and one for non-vegetarians. The plans will focus on addressing the abnormal findings in the blood report.\n\nI. 4-Week Vegetarian Diet Plan:\n a. Week 1\n Breakfast: Sprouted moong dal cheela with mint chutney.\n Lunch: Palak paneer with 2 multigrain rotis and a small bowl of cucumber raita.\n Dinner: Vegetable and quinoa khichdi with a side of carrot and beetroot salad.\n b. Week 2\n Breakfast: Besan (gram flour) and vegetable (spinach, fenugreek) uttapam with sambar.\n Lunch: Mixed vegetable curry (beans, carrots, peas, cauliflower) with 2 multigrain rotis and a bowl of curd.\n Dinner: Daliya (broken wheat) upma with mixed vegetables (carrots, peas, beans).\n c. Week 3\n Breakfast: Methi (fenugreek) paratha with a dollop of homemade white butter.\n Lu